<a href="https://colab.research.google.com/github/joshjarvey/Coursera_Capstone/blob/master/Capstone_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np


In [20]:
  #This data was gathered from an open public source via the City of Madison's Database:http://data-cityofmadison.opendata.arcgis.com/datasets/neighborhood-plans
  #Since the database is an ARCGIS database, it only contained SHAPE data.
  #The data was downloaded as a "SHAPE" file and converted to latitude and longitude using the free tool here:https://mygeodata.cloud/converter/shp-to-latlong

  #read in the .csv data, and drop the columns that are not needed for the analysis of this project
df_msn_neighborhood = pd.read_csv('/tmp/Neighborhood_Plans.csv').drop(['OBJECTID_1','OBJECTID','Status','Year','SHAPE_Leng','Link','ShapeSTAre','ShapeSTLen'], axis=1)

    #rename the columns for easier referencing in the project.
df_msn_neighborhood.rename(columns={'Y':'Latitude','X':'Longitude','PlanName':'Neighborhood'}, inplace=True)


,Longitude,Latitude,Neighborhood
0,-89.341014,43.098683,Schenk-Atwood-Starkweather-Worthington Park Ne...
1,-89.372108,43.086926,Tenney-Lapham-Old Market Neighborhood Plan
2,-89.358018,43.087919,Marquette-Schenk-Atwood Neighborhood Plan
3,-89.409940,43.061988,Brittingham-Vilas Neighborhood Plan
4,-89.390866,43.054009,Bay Creek Neighborhood Plan
5,-89.346647,43.048048,Broadway-Simpson-Waunona Neighborhood Plan
6,-89.456386,43.027492,Allied-Dunn's Marsh Neighborhood Plan
7,-89.329012,43.113940,Carpenter-Ridgeway-Hawthorne-Truax Neighborhoo...
8,-89.488393,43.045119,Southwest Neighborhood Plan
9,-89.384803,43.051256,South Madison Neighborhood Plan
